In [37]:
# !pip3 install langchain_community

In [38]:
from langchain_community.llms import Ollama

In [39]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nThe first man to walk on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first person to set foot on the moon.'

In [40]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

"\nOf course! I'd be happy to help you categorize your expenses. Here are the categories for each of the expenses you provided:\n\nTaxi Utrecht - Transportation\nMinisterie van Justitie en Veiligheid - Government\nEtos AMSTERDAM NLD - Food and Beverage\nBistro Bar Amsterdam - Food and Beverage\n\nI hope this helps! Let me know if you have any other questions."

### Read transaction data

In [3]:
# Read the transactions_2022_2023.csv file
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (INR)
0,01/14/2022,Audible Uk AdblCo/Pymt Gbr,Expense,11.00
1,01/15/2022,Classpass* Monthly,Expense,30.08
2,01/16/2022,Amazon Lux,Expense,24.11
3,01/31/2022,Salary income,Income,796911.67
4,02/28/2022,Salary income,Income,1.00


In [4]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [5]:
unique_transactions[1:10]

array(['Classpass* Monthly', 'Amazon Lux', 'Salary income', 'Tesco Breda',
       'Apple Services', 'Taxi Utrecht', 'Blogging', 'Tikkie',
       'Freelancing'], dtype=object)

### Categorise bank transactions with Llama2

In [44]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [45]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [46]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Of course! Here are the expenses with appropriate categories:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food', '4. Taxi Utrecht - Transportation', '5. Etos AMSTERDAM NLD - Groceries', '6. Bistro Bar Amsterdam - Dining']


,Transaction vs category,Transaction,Category
0,,,None
1,Of course! Here are the expenses with appropri...,Of course! Here are the expenses with appropri...,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
5,3. ISS Catering Services De Meern - Food,3. ISS Catering Services De Meern,Food
6,4. Taxi Utrecht - Transportation,4. Taxi Utrecht,Transportation
7,5. Etos AMSTERDAM NLD - Groceries,5. Etos AMSTERDAM NLD,Groceries
8,6. Bistro Bar Amsterdam - Dining,6. Bistro Bar Amsterdam,Dining


In [47]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

In [ ]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Sure! Here are the appropriate categories for ...,Sure! Here are the appropriate categories for ...,None
1,,,None
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Food Delivery,4. Vishandel Sier Amsterdam,Food Delivery
6,5. Selling Paintings - Art & Design,5. Selling Paintings,Art & Design
7,6. Spotify AB by Adyen - Entertainment,6. Spotify AB by Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Shopping,7. Tk Maxx Amsterdam Da,Shopping
9,8. Consulting - Professional Services,8. Consulting,Professional Services


In [ ]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [ ]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Taxes', 'Groceries', 'Housing', 'Food Delivery',
       'Art & Design', 'Entertainment', 'Shopping',
       'Professional Services', 'Charity Donations', 'Transportation',
       'Health & Beauty', 'Fitness & Sports', 'Home Improvement',
       'Dining Out', 'Personal Services', 'Financial Services',
       'Personal Activities', 'Tech & Electronics', 'Online Shopping',
       'Personal Income'], dtype=object)

In [ ]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [ ]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

/var/folders/4j/f6pm1l2x3r1bws0sscxxxtkw0000gn/T/ipykernel_78994/2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Food Delivery,4. Vishandel Sier Amsterdam,Food and Drinks
6,5. Selling Paintings - Art & Design,5. Selling Paintings,Art & Design
7,6. Spotify AB by Adyen - Entertainment,6. Spotify AB by Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Shopping,7. Tk Maxx Amsterdam Da,Shopping
9,8. Consulting - Professional Services,8. Consulting,Services
10,9. Aidsfonds - Charity Donations,9. Aidsfonds,Charity Donations
11,10. TLS BV Inz Ov-Chipkaart - Transportation,10. TLS BV Inz Ov-Chipkaart,Transportation


In [ ]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (INR),Transaction vs category,Transaction,Category
0,12/30/2023,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,12/30/2023,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,12/30/2023,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,12/30/2023,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,12/29/2023,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,12/29/2023,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,12/23/2023,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,12/22/2023,Consulting,Income,541.57,NaN,NaN,NaN
8,12/22/2023,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,12/20/2023,Consulting,Income,2641.93,NaN,NaN,NaN


In [ ]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)